In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb; sb.set()

# Hierarchical Indexing
1. Pandas Data Structure that can handle high dimensional data
    - `MultiIndex`
    - `Panel`
    - `Panel4D`
2. Extra dimensions: 
    - Multi-indexed Series to Dataframe: `stack()`
    - Dataframe back to Multi-indexed Series: `unstack()`
3. Five Methods to Create MultiIndex
4. Giving index names
5. Set and reset index
6. Sort Indices so that slicing can be performed
7. Aggregating across multiIndex
    - `.apply(axis=1)`: apply the function across columns for each row
    - `.concat(axis=1)`: concatenate columns
    - `.dropna(axis=1)`: drop columns

## Create MultiIndex
1. `from_arrays()`
2. `from_tuples()`
3. `from_product()`
4. `pd.MultiIndex(levels = [], codes = [])`
5. `dataframe.set_index([])`

In [2]:
# Inefficient Way. Can index or slice, but cannot select all values from 2010!
index = [('California', 2000), ('California', 2010),
                ('New York', 2000), ('New York', 2010),
                ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
              18976457, 19378102,
              20851820, 25145561]
pop = pd.Series(populations, index=index)
print(pop[[i for i in pop.index if i[1] == 2010]])
pop

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64


(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
# the parameter accepts only one list, which contains lists
# Four methods to create multiIndex
index = pd.MultiIndex.from_tuples([('California', 2000), ('California', 2010),
        ('New York', 2000), ('New York', 2010),
        ('Texas', 2000), ('Texas', 2010)])
index = pd.MultiIndex.from_product([['California', 'New York', 'Texas'], [2000, 2010]])
index = pd.MultiIndex.from_arrays([['California', 'California', 'New York', 'New York', 'Texas', 'Texas'],
                                [2000, 2010, 2000, 2010, 2000, 2010]])
index = pd.MultiIndex(levels = [['California', 'New York', 'Texas'], [2000, 2010]], codes = [[0,0,1,1,2,2], [0,1,0,1,0,1]])
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [4]:
# Name each level of multiIndex
pop = pop.reindex(index)
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [5]:
pop_df = pop.reset_index(name = 'population') # destroy multiIndex, converting them into columns
pop_df

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [6]:
# Method 5 to create index
pop_df.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [7]:
pop[:, 2010] # select * from first column, 2010 from second column

state
California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [8]:
# to add an additional column, it is just the same
pop_df = pop.unstack()
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
state      year                   
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [9]:
f_u18 = pop_df['under18']/ pop_df['total']
f_u18

state       year
California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [10]:
f_u18.unstack() # level = 1. Inner level by default

year,2000,2010
state,,
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [11]:
f_u18.unstack(level = 0) # level 0 starts from outer level

state,California,New York,Texas
year,,,
2000,0.273594,0.247010,0.283251
2010,0.249211,0.222831,0.273568


# Columns can have MultiIndex!

In [16]:
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

data = np.round(np.random.randn(4, 6), 1)
data[:, ::2]*=10
data += 37

health_data = pd.DataFrame(data, index = index, columns = columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  37.5  30.0  38.5  23.0  36.1
     2      27.0  37.1  28.0  36.3  28.0  35.8
2014 1      17.0  37.0  20.0  36.0  39.0  37.0
     2      47.0  35.9  41.0  37.1  40.0  38.1

# Indexing and Slicing

In [17]:
# Series
display(pop['California', 2000]) # Full Indexing
display(pop['California']) # Partial Indexing: outer level
display(pop[:, 2000]) # Indexing inner level: inner level

display(pop['California':'New York']) # Slicing
display(pop[pop > 22000000]) # Masking
display(pop[['California', 'Texas']]) # Fancy Indexing

33871648

year
2000    33871648
2010    37253956
dtype: int64

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

In [60]:
# DataFrame
# Indexing
display(health_data['Guido', 'HR']) # columns are primary in DataFrames
display(health_data.loc[:, ('Bob', 'HR')]) # only that column. Hence a Series

# Slicing
display(health_data.iloc[:2, :2]) # gets 2x2 array, irrespective of the multi-indexes
idx = pd.IndexSlice
display(health_data.loc[idx[:,1], idx[:, 'HR']]) # Slicing requires IndexSlice

year  visit
2013  1        35.0
      2        39.0
2014  1        20.0
      2        26.0
Name: (Guido, HR), dtype: float64

year  visit
2013  1        27.0
      2        54.0
2014  1        49.0
      2        29.0
Name: (Bob, HR), dtype: float64

subject      Bob      
type          HR  Temp
year visit            
2013 1      27.0  38.2
     2      54.0  37.1

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,27.0,35.0,40.0
2014,1,49.0,20.0,43.0


# Sort Indices
**slicing** operations will mostly fail if indices are not sorted
- Hence, use `sort_index()` or `sortlevel()`

In [9]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1,2]])
data = pd.Series(np.random.randn(6), index = index)
data.index.names = ['char', 'int']
data

char  int
a     1     -0.091224
      2     -1.028061
c     1      1.085957
      2      2.146662
b     1      0.937133
      2     -1.672941
dtype: float64

In [10]:
try:
    data['a':'b']
except KeyError as e:
    print(e)
    data = data.sort_index()
    print(data['a':'b'])

'Key length (1) was greater than MultiIndex lexsort depth (0)'
char  int
a     1     -0.091224
      2     -1.028061
b     1      0.937133
      2     -1.672941
dtype: float64


# Aggregations on Multi-Indices

In [18]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  37.5  30.0  38.5  23.0  36.1
     2      27.0  37.1  28.0  36.3  28.0  35.8
2014 1      17.0  37.0  20.0  36.0  39.0  37.0
     2      47.0  35.9  41.0  37.1  40.0  38.1

In [19]:
# Average measurement in the visits each year
data_mean = health_data.mean(level = 'year')

C:\Users\oongj\AppData\Local\Temp\ipykernel_22392\522248412.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  data_mean = health_data.mean(level = 'year')


subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     34.5  37.30  29.0  37.40  25.5  35.95
2014     32.0  36.45  30.5  36.55  39.5  37.55

In [42]:
# Mean HR and Temp across years
data_mean.mean(axis = 1, level = 'type')

type,HR,Temp
year,,
2013,37.000000,37.30
2014,38.833333,37.55
